# Como criar um projeto com uma fonte de dados mais proxima do existente no dia a dia?

## Bibliotecas

> requests → fazer requisções e retornar o conteúdo HTML do site

> BeautifulSoup → Manipular conteúdo HTML (utilizando as relações de parentesco)

> pandas → Manipular dados e criação de arquivo

> tqdm → Visulização de barra de progresso

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from tqdm import tqdm

## Criação da função get_data

1. Requisição no site com número da página escolhida.
2. Criação de um objeto BeatifulSoup para a manipulação dos dados.
3. Obtenção de todas as `div` com a classe `data-list-item`
4. Iteração dentro de cada uma delas com as seguintes ações:
- Criação das variaveis dentro de um try para atribuir nulo se não encontrado o valor.
- Encontrar o elemento `strong` que tem o texto interno dentro do atributo `string`.
- Retornar o elemento pai dele, pois `o strong` esta dentro da `div` que eu desejava trabalhar.
- Retirar o texto que estava dentro do elemento `strong` para retornar apenas o valor, retirar os \n, \t e espaços nas pontas.
5. Retorna cada elemento em tuplas

In [30]:
def get_data(pg):

  '''
    A partir do número da página do stie da camara de capivari ele retorna

    pg                 -> Numero da página,
    formato das tuplas -> ( 'Protocolo', 'Data', 'Assunto', 'Situação', 'Regime', 'Autoria')
  '''

  res = requests.get(f'https://capivari.siscam.com.br/Documentos/Pesquisa?Pesquisa=Simples&id=80&pagina={pg}&Modulo=8&Documento=0&Numeracao=Documento&NumeroInicial=&AnoInicial=')
  bs = BeautifulSoup(res.content, 'html.parser')
  itens = []
  for item in bs.find_all('div', class_ = 'data-list-item'):

    try:
      protocolo = item.find('strong', string = 'Protocolo:').parent.text.replace('\nProtocolo: ',"").replace('\r\n','').strip()
    except:
      protocolo = ''

    try:
      data = item.find('strong', string = 'Data:').parent.text.replace('\nData: ',"").replace('\r\n','').strip()
    except:
      data = ''

    try:
      assunto = item.find('strong', string = 'Assunto:').parent.text.replace('\nAssunto: ',"").replace('\r\n','').strip()
    except:
      assunto = ''

    try:
      situacao = item.find('strong', string = 'Situação:').parent.text.replace('\nSituação: ',"").replace('\r\n','').strip()
    except:
      situacao = ''

    try:
      regime = item.find('strong', string = 'Regime:').parent.text.replace('\nRegime: ',"").replace('\r\n','').strip()
    except:
      regime = ''

    try:
      autoria = item.find('strong', string = 'Autoria:').parent.text.replace('\nAutoria: ',"").replace('\r\n','').strip()
    except:
      autoria = ''

    itens.append(
        (
            protocolo,
            data,
            assunto,
            situacao,
            regime,
            autoria
        )
        )
  return itens

### Exemplo de formato de cada item

```


<div class="data-list-item data-list-striped data-list-hover">                     
  <h4><a href="/Documentos/Documento/39952" target="_self" title="Requerimento Nº 28/2024">Requerimento Nº 28/2024</a></h4>

  <div class="row">
    <p class="col-md-4">
        <strong>Data:</strong> 20/02/2024
    </p>
    <p class="col-md-4">
        <strong>Protocolo:</strong> 00594/2024
    </p>
  </div>
  
  <div class="row">
    <p class="col-md-4">
        <strong>Situação:</strong> Protocolada
    </p>
    <p class="col-md-4">
        <strong>Regime:</strong> Ordinário
    </p>
  </div>

  <div class="row">
    <p class="col-md-12">
        <strong>Autoria:</strong> MARCOS SABINO FERNANDES
    </p>
  </div>

  <div class="row">
    <p class="col-md-12">
        <strong>Assunto:</strong> Solicita informações sobre nebulização de inseticida no município. Entre outras informações.
    </p>
  </div>

  <ul class="list-inline hidden-print">
    <li>
        <a class="btn btn-primary" href="/Documentos/ListarArquivosPdf/39952" title="Processo completo" target="_blank"><span aria-hidden="true" class="fa fa-file-text-o fa-lg"></span></a>
      </li>
    <li class="list-inline-padding"><a href="/arquivo?Id=14533" title="2024req028" target="_blank" class="list-inline-padding"><span aria-hidden="true" class="fa fa-2x fa-file-word-o"></span></a></li>
  </ul>

</div>
```

## Criação da função create_df

1. Validação dos números de começo e fim.
2. iteração nesses números acumulando dentro de uma lista.
3. Criação de um dataframe com a união de todas as listas.

In [33]:
def create_df(start, end):

  '''

    A partir do número da página inicial e final ele retorna um DataFrame com todas as informações desejadas.

    start -> Numeração inicial da página
    end   -> Numeração final da página

  '''

  assert start >0 and start<=end, 'Error on start/end values'

  dfs = []

  for c in tqdm(range(start, end+1)):
    dfs.append(pd.DataFrame(get_data(c), columns = ['Protocolo', 'Data', 'Assunto', 'Situação', 'Regime', 'Autoria']))

  return pd.concat(dfs, axis=0, ignore_index=True)

## Execução

## Criação da base

In [34]:
join = create_df(1,100)

100%|██████████| 100/100 [02:47<00:00,  1.67s/it]


In [35]:
display(join.head())

,Protocolo,Data,Assunto,Situação,Regime,Autoria
0,00594/2024,20/02/2024,Solicita informações sobre nebulização de inse...,Protocolada,Ordinário,MARCOS SABINO FERNANDES
1,00593/2024,20/02/2024,Moção de Repúdio ao Excelentíssimo Presidente ...,Protocolada,Ordinário,JOSÉ EDUARDO DE CAMPOS PACHECO
2,00580/2024,19/02/2024,Solicita melhorias nas ruas do bairro Cancian,Protocolada,Ordinário,THIAGO JUNIOR ANESIO BRAGGION
3,00533/2024,19/02/2024,Solicita troca de lâmpadas queimadas na rodovi...,Protocolada,Ordinário,DENILTON ROCHA DOS SANTOS
4,00526/2024,19/02/2024,Solicita melhorias na ponte (Pontinha do SAAE)...,Protocolada,Ordinário,RUAN HENRIQUE PELLEGRINI BRAGGION


## Registro da base

In [36]:
join.to_csv('raw_data.csv', index = False)